In [6]:
devtools::install_github("WorldWeatherAttribution/rwwa")
suppressMessages(library("rwwa"))


── R CMD build ─────────────────────────────────────────────────────────────────
/bin/sh: which: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `which'
/bin/sh: ml: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `ml'
/bin/sh: scl: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `scl'
/bin/sh: module: line 1: syntax error: unexpected end of file
/bin/sh: error importing function definition for `module'
sh: which: line 1: syntax error: unexpected end of file
sh: error importing function definition for `which'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
sh: scl: line 1: syntax error: unexpected end of file
sh: error importing function definition for `scl'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: which: line 1: syntax erro

## Compare to Climate Explorer synthesis

In [8]:
for(fnm in list.files("data", full.names = F)) {
    
    fnm <- gsub(".csv", "", fnm)
    # load the data
    df <- read.csv(paste0("data/",fnm,".csv"), row.names = "dataset")

    # split into obs & models, specify the type of synthesis
    obs_in <- df[df$src == "obs",-1]
    models_in <- df[df$src == "models",-1]
    synth_type <- gsub("_","",substr(fnm, nchar(fnm)-2, nchar(fnm)))

        # save in synthesis format
    synth_fnm <- paste0("to_cx/",fnm,".txt")

    obs_cx <- cbind(1980,2020, obs_in, 95,rownames(obs_in))
    model_cx <- cbind(1980,2020, models_in, 95,rownames(models_in))

    colnames(obs_cx) <- colnames(model_cx) <- c("#1", 2:ncol(obs_cx))
    write.table(rbind(obs_cx, model_cx), file = synth_fnm, quote = F, row.names = F)

    # load output from climate explorer
    cx_df <- read.table(paste0("cx_synth/cx-synth_",fnm,".txt"), col.names = c("y1", "y2", "est", "lower", "upper", "l_wb", "u_wb", "group", "model"))

    # run synthesis using R
    synth <- synthesis(obs_in = df[df$src == "obs",-1], models_in = df[df$src == "models",-1], synth_type = synth_type)
    synth_cx <- synthesis_cx(obs_in = df[df$src == "obs",-1], models_in = df[df$src == "models",-1], synth_type = synth_type)

    cx <- readLines(paste0("cx_synth/cx-synth_",fnm,".txt"))

    # get sig_obs parameter from CX output
    if(any(grep("rep", cx))) {
        cx_split <- suppressWarnings(sapply(strsplit(cx[grepl("rep", cx)], " +"), function(x) tryCatch(as.numeric(x))))
        cx_sigobs <- cx_split[!is.na(cx_split)]
    } else {
        cx_sigobs <- NA
    }

    if(any(grepl("model uncertainty is neglected", cx))) {
        cx_sigmod <- NA
    } else {
        cx_split <- suppressWarnings(sapply(strsplit(cx[grepl("model uncertainty", cx)], " +"), function(x) tryCatch(as.numeric(x))))
        cx_sigmod <- cx_split[!is.na(cx_split)]
    } 

    cx <- list("synth_type" = synth_type,
               "sig_obs" = cx_sigobs,
               "chi2/dof" = as.numeric(strsplit(cx[grepl("chi2", cx)], " +")[[1]][5]),
               "sig_mod" = cx_sigmod,
               "df" = read.table(paste0("cx_synth/cx-synth_",fnm,".txt"), col.names = c("y1", "y2", "est", "lower", "upper", "l_wb", "u_wb", "group", "model")))

    res <- rbind("R" = unlist(synth[c("chi2/dof", "sig_obs", "sig_mod")]), "cx" = c(cx[["chi2/dof"]], cx[["sig_obs"]], cx[["sig_mod"]]),
                "R_cx" = unlist(synth_cx[c("chi2/dof", "sig_obs", "sig_mod")]))

                                            
    ########################################################
    synth <- synth_cx
    ########################################################
                                            
                                            
    if(synth_type == "PR") log_axs = "xy" else log_axs = ""
    xylim <- range(unlist(synth$df[,-(1:2)]), na.rm = T)

    pdf(paste0("fig/bounds_",fnm,".pdf"), h = 6, w = 25); {
        layout(matrix(1:5, nrow = 1), widths = c(4,4,5,5,3)/6)
        par(oma = c(0,10,2,0))

        # First base plot
        par(mar = c(3,0.5,3,0.5))
        plot_synthesis(synth, main = "R")
        plot_synthesis(cx, hide_labels = T, main = "Climate explorer")

        mtext(fnm, side = 3, outer = T, font = 2)

        # check coloured bounds for each group
        par(mar = c(3,2,3,0.5))
        plot(synth$df[synth$df$group == "obs", "lower"], cx$df[cx$df$group == "1", "lower"], xlim = xylim, ylim = xylim, log = log_axs, col = "blue", pch = 21, lwd = 2,
            main = "Coloured bounds", xlab = "Computed in R", ylab = "ClimExp")
        points(synth$df[synth$df$group == "obs", "upper"], cx$df[cx$df$group == "1", "upper"], xlim = xylim, ylim = xylim, col = "blue", pch = 3, lwd = 2)

        points(synth$df[synth$df$group == "obs_synth", "lower"], cx$df[cx$df$group == "2", "lower"], xlim = xylim, ylim = xylim, col = "blue4", pch = 21, lwd = 2)
        points(synth$df[synth$df$group == "obs_synth", "upper"], cx$df[cx$df$group == "2", "upper"], xlim = xylim, ylim = xylim, col = "blue4", pch = 3, lwd = 2)

        points(synth$df[synth$df$group == "models", "lower"], cx$df[cx$df$group == "3", "lower"], xlim = xylim, ylim = xylim, col = "red2", pch = 21, lwd = 2)
        points(synth$df[synth$df$group == "models", "upper"], cx$df[cx$df$group == "3", "upper"], xlim = xylim, ylim = xylim, col = "red2", pch = 3, lwd = 2)

        points(synth$df[synth$df$group == "model_synth", "lower"], cx$df[cx$df$group == "4", "lower"], xlim = xylim, ylim = xylim, col = "red4", pch = 21, lwd = 2)
        points(synth$df[synth$df$group == "model_synth", "upper"], cx$df[cx$df$group == "4", "upper"], xlim = xylim, ylim = xylim, col = "red4", pch = 3, lwd = 2)

        points(synth$df[synth$df$group == "synth", "lower"], cx$df[cx$df$group == "5", "lower"], xlim = xylim, ylim = xylim, col = "magenta", pch = 21, lwd = 2)
        points(synth$df[synth$df$group == "synth", "upper"], cx$df[cx$df$group == "5", "upper"], xlim = xylim, ylim = xylim, col = "magenta", pch = 3, lwd = 2)
        abline(0,1, lty = 3) 

        # check white bounds for each group
        plot(synth$df[synth$df$group == "obs", "l_wb"], cx$df[cx$df$group == "1", "l_wb"], xlim = xylim, ylim = xylim, log = log_axs, col = "blue", pch = 21, lwd = 2,
            main = "White bounds", xlab = "Computed in R", ylab = "ClimExp")
        points(synth$df[synth$df$group == "obs", "u_wb"], cx$df[cx$df$group == "1", "u_wb"], xlim = xylim, ylim = xylim, col = "blue", pch = 3, lwd = 2)

        points(synth$df[synth$df$group == "models", "l_wb"], cx$df[cx$df$group == "3", "l_wb"], xlim = xylim, ylim = xylim, col = "red2", pch = 21, lwd = 2)
        points(synth$df[synth$df$group == "models", "u_wb"], cx$df[cx$df$group == "3", "u_wb"], xlim = xylim, ylim = xylim, col = "red2", pch = 3, lwd = 2)

        points(synth$df[synth$df$group == "synth", "l_wb"], cx$df[cx$df$group == "5", "l_wb"], xlim = xylim, ylim = xylim, col = "magenta", pch = 21, lwd = 2)
        points(synth$df[synth$df$group == "synth", "u_wb"], cx$df[cx$df$group == "5", "u_wb"], xlim = xylim, ylim = xylim, col = "magenta", pch = 3, lwd = 2)
        abline(0,1, lty = 3) 


        legend("bottomright", c("Lower bound", "Upper bound"), pch = c(21,3))

        # plot table of results
        frame()
        vps <- baseViewports()
        pushViewport(vps$inner, vps$figure, vps$plot)

        grob <- tableGrob(round(res,3),  vp = viewport(x = 0.1, y = 0.75))
        grid.draw(grob)

    }; dev.off()
}

## troubleshooting & dev

In [2]:
getsynmean <- function(data, sig_mod = 0) {

  # calculate weight for each model based on inverse variance
  w = 1/(((data$upper - data$lower)/(2*1.96))^2 + sig_mod^2)
  w1 = sum(w)

  # weighted mean
  s1 <- sum(w*data$est) / w1

  # get weighted interval by adding variances
  sig_lower = sqrt(sum(w * (((data$est - data$lower)/1.96)^2 + sig_mod^2)) / w1)
  sig_upper = sqrt(sum(w * (((data$est - data$upper)/1.96)^2 + sig_mod^2)) / w1)

  return(setNames(s1 + c(0, -1.96*sig_lower, +1.96*sig_upper), c("est", "lower", "upper")))
}


################################################################################################################################
#' Support function to calculate the chi^2 function used to estimate sig_mod
#'
#' @param data Data.frame with one row per climate model, with columns containing best estimate and upper and lower bound of 95pc confidence interval for the quantity of interest
#' @param sig_mod Scalar: model representation error. Default value is 0
#'
#' @return Vector of best estimate and 95pcconfidence bounds for weighted model mean, with and without model representation error
#'
#' @keywords internal
#' @export
#'
getsynchi2 <- function(data, sig_mod = 0) {

  # function to be minimized by finding sig_mod such that chi^2/mdof ~= 1

  # get best estimate of weighted mean
  s1 <- getsynmean(data, sig_mod)["est"]

  # compute chi2 by converting model intervals to standard deviations & adding sig_mod adjustment
  chi2 <- sum(apply(data[,c("est", "lower", "upper")], 1, function(x) {
    if(x["est"] > s1) {
      (x["est"] - s1)^2 / (((x["est"] - x["lower"])/1.96)^2 + sig_mod^2)
    } else {
      (x["est"] - s1)^2 / (((x["est"] - x["upper"])/1.96)^2 + sig_mod^2)
    }
  }))
  return(chi2)
}


################################################################################################################################
#' Produce synthesis of attribution results from observations and climate models
#'
#' @param obs_in Data.frame with one row per observational dataset, with columns "est", "lower" and "upper" giving the best estimate and lower and upper bounds for the quantity of interest. Dataset names should be given as rownames. Default is NA (no observations), in which case only the model synthesis is carried out.
#' @param models_in Data.frame with one row per climate model, with columns "est", "lower" and "upper" giving the best estimate and lower and upper bounds for the quantity of interest. Model names should be given as rownames.
#' @param synth_type String defining the type of synthesis to carry out. Options are 'abs' (absolute changes); 'rel' (percentage changes); and 'PR' (probability ratios). Default is 'abs'.
#'
#' @return List containing synth_type; sig_obs (scalar value indicating obs representation error); sig_mod (scalar value indicating model representation error); chi2/dof (initial estimate of ratio of chi^2 to model DOF); and df, a data.frame containing the synthesised results.
#'
#' @export
#'
synthesis <- function(obs_in = NA, models_in, synth_type = "abs") {

  if(is.na(unlist(obs_in))[1]) {
    no_obs <- T
    # create a dummy dataframe to avoid having to rewrite everything twice
    obs_in <- data.frame("est" = 0, "lower" = 0, "upper" = 0)
    rownames(obs_in) <- "dummy"
  } else {
    no_obs <- F
  }

  # relabel the data for easier reference later
  colnames(obs_in) <- colnames(models_in) <- c("est", "lower", "upper")

  if(!("model" %in% colnames(obs_in))) obs_in$model <- rownames(obs_in)
  if(!("model" %in% colnames(models_in))) models_in$model <- rownames(models_in)

  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

  if(!synth_type %in% c("abs", "rel", "PR")) {
    cat(paste0("Synthesis type '",synth_type,"' not implemented - must be abs, rel or PR"), "\n")
  }

  if(synth_type == "PR") {
    obs_in[,c("est", "lower", "upper")] <- log(obs_in[,c("est", "lower", "upper")])
    models_in[,c("est", "lower", "upper")] <- log(models_in[,c("est", "lower", "upper")])
  } else if(synth_type == "rel") {
    obs_in[,c("est", "lower", "upper")] <- log(1+obs_in[,c("est", "lower", "upper")]/100)
    models_in[,c("est", "lower", "upper")] <- log(1+models_in[,c("est", "lower", "upper")]/100)
  }

  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # get mean of intervals to estimate natural variability component

  # compute representation error from scatter of mean ($\sigma_{rep}$ in the paper)
  nobs = nrow(obs_in)
  obs <- apply(obs_in[,c("est", "lower", "upper"),drop = F], 2, mean)
  if(nobs == 1) {
    sig_obs = 0
  } else {
    s2 = sum((obs_in$est - obs[1])^2)
    sig_obs = sqrt(s2/(nobs-1))
  }

  # add representation error to individual observations
  obs_in$l_wb <- obs_in$est - sqrt((obs_in$est - obs_in$lower)**2 + (1.96*sig_obs)**2)
  obs_in$u_wb <- obs_in$est + sqrt((obs_in$est - obs_in$upper)**2 + (1.96*sig_obs)**2)

  # apply representation error to obs synthesis
  # we're working with confidence intervals here, so we extend them by adding (1.96sig_obs)^2 in quadrature
  obs[2] <- obs[1] - sqrt( (obs[1] - obs[2])**2 + (1.96*sig_obs)**2 )
  obs[3] <- obs[1] + sqrt( (obs[1] - obs[3])**2 + (1.96*sig_obs)**2 )

  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # get initial estimate of model mean & calculate chi^2
  chi2 <- getsynchi2(models_in, sig_mod = 0)
  mdof <- nrow(models_in)-1

  if ( chi2/mdof > 1 ) {
    # find sig_mod such that chi^2/dof = 1
    sig_mod <- optim(0, function(x) {(getsynchi2(models_in, sig_mod = x) - (nrow(models_in)-1))^2},
                     method = "Brent", lower = 0, upper = 5)$par
  } else {
    sig_mod <- 0
  }

  # get weighted model mean
  models <- getsynmean(models_in, sig_mod = sig_mod)

  # add representation error to individual models
  models_in$l_wb <- models_in$est - sqrt((models_in$est - models_in$lower)**2 + (1.96*sig_mod)**2)
  models_in$u_wb <- models_in$est + sqrt((models_in$est - models_in$upper)**2 + (1.96*sig_mod)**2)

  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # weighted mean of models & obs (coloured bar)
  w_obs <- unname((obs["upper"] - obs["lower"])^{-2})
  w_mod <- unname((models["upper"] - models["lower"])^{-2})

  wmean <- (w_obs * obs["est"] + w_mod * models["est"]) / (w_obs + w_mod)

  # get weighted interval by averaging variances
  sig_lower = sqrt((w_obs * ((obs["est"] - obs["lower"])/1.96)^2 + w_mod * ((models["est"] - models["lower"])/1.96)^2) / (w_obs + w_mod))
  sig_upper = sqrt((w_obs * ((obs["est"] - obs["upper"])/1.96)^2 + w_mod * ((models["est"] - models["upper"])/1.96)^2) / (w_obs + w_mod))
  synth <- setNames(c(wmean, wmean - 1.96*sig_lower, wmean + 1.96*sig_upper), c("est", "lower", "upper"))

  # unweighted mean of obs and models
  umean <- (obs["est"] +  models["est"]) / 2
  synth["l_wb"] <- umean - sqrt(((obs["est"]-obs["lower"])^2 + (models["est"]-models["lower"])^2)/2)
  synth["u_wb"] <- umean + sqrt(((obs["est"]-obs["upper"])^2 + (models["est"]-models["upper"])^2)/2)

  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # add group labels for easier plotting
  obs_in <- cbind(obs_in, "group" = "obs")
  obs <- data.frame(t(c("model" = "Observations", "group" = "obs_synth", obs)))
  models_in <- cbind(models_in, "group" = "models")
  models <- data.frame(t(c("model" = "Models", "group" = "model_synth", models)))
  synth <- data.frame(t(c("model" = "Synthesis", "group" = "synth", synth)))

  # combine all the data together in one dataframe
  res <- rbind.fill(obs_in, obs, models_in, models, synth)[,c("group", "model", "est", "lower", "upper", "l_wb", "u_wb")]
  for(cnm in c("est", "lower", "upper", "l_wb", "u_wb")) { res[,cnm] <- as.numeric(res[,cnm]) }

  # if only dummy obs, remove
  if(no_obs) {
    # drop all rows that don't relate to models
    res <- res[grepl("model", res$group),]
    sig_obs <- NA
  }

  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # reverse any transformations applied
  if(synth_type == "PR") {
    res[,c("est", "lower", "upper", "l_wb", "u_wb")] <- exp(res[,c("est", "lower", "upper", "l_wb", "u_wb")])
    sig_obs <- exp(sig_obs)
    sig_mod <- exp(sig_mod)
    umean <- exp(umean)
  } else if(synth_type == "rel") {
    res[,c("est", "lower", "upper", "l_wb", "u_wb")] <- 100*(exp(res[,c("est", "lower", "upper", "l_wb", "u_wb")])-1)
    sig_obs <- 100*(exp(sig_obs)-1)
    sig_mod <- 100*(exp(sig_mod)-1)
    umean <- 100*(exp(umean)-1)
  }

  return(list(synth_type = synth_type, sig_obs = sig_obs, "chi2/dof" = chi2 / mdof, sig_mod = sig_mod, df = res, uw_mean = umean))
}

In [3]:
# 288: sig_obs shouldn't be x2 (since this outputs the wrong value of sig_obs)
# 289-296: should use 1.96*sig_obs in all these expressions

getsynmean_cx <- function(data, sig_mod = 0) {

  # calculate weight for each model based on inverse variance
  w = 1/(((data$upper - data$lower)/(2*1.96))^2 + sig_mod^2)
  w1 = sum(w)

  # weighted mean
  s1 <- sum(w*data$est) / w1

  # get weighted interval by adding variances
  sig_lower = sqrt(sum(w * (((data$est - data$lower)/1.96)^2 + sig_mod^2)) / w1)
  sig_upper = sqrt(sum(w * (((data$est - data$upper)/1.96)^2 + sig_mod^2)) / w1)

  return(setNames(s1 + c(0, -1.96*sig_lower, +1.96*sig_upper), c("est", "lower", "upper")))
}


################################################################################################################################
#' Support function to calculate the chi^2 function used to estimate sig_mod
#'
#' @param data Data.frame with one row per climate model, with columns containing best estimate and upper and lower bound of 95pc confidence interval for the quantity of interest
#' @param sig_mod Scalar: model representation error. Default value is 0
#'
#' @return Vector of best estimate and 95pcconfidence bounds for weighted model mean, with and without model representation error
#'
#' @keywords internal
#' @export
#'
getsynchi2_cx <- function(data, sig_mod = 0) {

  # function to be minimized by finding sig_mod such that chi^2/mdof ~= 1

  # get best estimate of weighted mean
  s1 <- getsynmean_cx(data, sig_mod)["est"]

  # compute chi2 by converting model intervals to standard deviations & adding sig_mod adjustment
  chi2 <- sum(apply(data[,c("est", "lower", "upper")], 1, function(x) {
    if(x["est"] > s1) {
      (x["est"] - s1)^2 / (((x["est"] - x["lower"])/1.96)^2 + sig_mod^2)
    } else {
      (x["est"] - s1)^2 / (((x["est"] - x["upper"])/1.96)^2 + sig_mod^2)
    }
  }))
  return(chi2)
}


synthesis_cx <- function(obs_in = NA, models_in, synth_type = "abs") {

  if(is.na(unlist(obs_in))[1]) {
    no_obs <- T
    # create a dummy dataframe to avoid having to rewrite everything twice
    obs_in <- data.frame("est" = 0, "lower" = 0, "upper" = 0)
    rownames(obs_in) <- "dummy"
  } else {
    no_obs <- F
  }

  # relabel the data for easier reference later
  colnames(obs_in) <- colnames(models_in) <- c("est", "lower", "upper")

  if(!("model" %in% colnames(obs_in))) obs_in$model <- rownames(obs_in)
  if(!("model" %in% colnames(models_in))) models_in$model <- rownames(models_in)

  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

  if(!synth_type %in% c("abs", "rel", "PR")) {
    cat(paste0("Synthesis type '",synth_type,"' not implemented - must be abs, rel or PR"), "\n")
  }

  if(synth_type == "PR") {
    obs_in[,c("est", "lower", "upper")] <- log(obs_in[,c("est", "lower", "upper")])
    models_in[,c("est", "lower", "upper")] <- log(models_in[,c("est", "lower", "upper")])
  } else if(synth_type == "rel") {
    obs_in[,c("est", "lower", "upper")] <- log(1+obs_in[,c("est", "lower", "upper")]/100)
    models_in[,c("est", "lower", "upper")] <- log(1+models_in[,c("est", "lower", "upper")]/100)
  }

  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # get mean of intervals to estimate natural variability component

  # compute representation error from scatter of mean ($\sigma_{rep}$ in the paper)
  nobs = nrow(obs_in)
  obs <- apply(obs_in[,c("est", "lower", "upper"),drop = F], 2, mean)
  if(nobs == 1) {
    sig_obs = 0
  } else {
    s2 = sum((obs_in$est - obs[1])^2)
    sig_obs = 2*sqrt(s2/(nobs-1))
  }
    
  # add representation error to individual observations
  obs_in$l_wb <- obs_in$est - sqrt((obs_in$est - obs_in$lower)**2 + (sig_obs)**2)
  obs_in$u_wb <- obs_in$est + sqrt((obs_in$est - obs_in$upper)**2 + (sig_obs)**2)

  # apply representation error to obs synthesis
  # we're working with confidence intervals here, so we extend them by adding (1.96sig_obs)^2 in quadrature
  obs[2] <- obs[1] - sqrt( (obs[1] - obs[2])**2 + (sig_obs)**2 )
  obs[3] <- obs[1] + sqrt( (obs[1] - obs[3])**2 + (sig_obs)**2 )

  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # get initial estimate of model mean & calculate chi^2
  chi2 <- getsynchi2_cx(models_in, sig_mod = 0)
  mdof <- nrow(models_in)-1

  if ( chi2/mdof > 1 ) {
    # find sig_mod such that chi^2/dof = 1
    sig_mod <- optim(0, function(x) {(getsynchi2_cx(models_in, sig_mod = x) - (nrow(models_in)-1))^2},
                     method = "Brent", lower = 0, upper = 5)$par
  } else {
    sig_mod <- 0
  }

  # get weighted model mean
  models <- getsynmean_cx(models_in, sig_mod = sig_mod)

  # add representation error to individual models
  models_in$l_wb <- models_in$est - sqrt((models_in$est - models_in$lower)**2 + (sig_mod)**2)
  models_in$u_wb <- models_in$est + sqrt((models_in$est - models_in$upper)**2 + (sig_mod)**2)

  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # weighted mean of models & obs (coloured bar)
  w_obs <- unname((obs["upper"] - obs["lower"])^{-2})
  w_mod <- unname((models["upper"] - models["lower"])^{-2})

  wmean <- (w_obs * obs["est"] + w_mod * models["est"]) / (w_obs + w_mod)

  # get weighted interval by averaging variances
  sig_lower = sqrt((w_obs * ((obs["est"] - obs["lower"])/1.96)^2 + w_mod * ((models["est"] - models["lower"])/1.96)^2) / (w_obs + w_mod))
  sig_upper = sqrt((w_obs * ((obs["est"] - obs["upper"])/1.96)^2 + w_mod * ((models["est"] - models["upper"])/1.96)^2) / (w_obs + w_mod))
  synth <- setNames(c(wmean, wmean - 1.96*sig_lower, wmean + 1.96*sig_upper), c("est", "lower", "upper"))

  # unweighted mean of obs and models
  umean <- (obs["est"] +  models["est"]) / 2
  synth["l_wb"] <- umean - sqrt(((obs["est"]-obs["lower"])^2 + (models["est"]-models["lower"])^2)/2)
  synth["u_wb"] <- umean + sqrt(((obs["est"]-obs["upper"])^2 + (models["est"]-models["upper"])^2)/2)

  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # add group labels for easier plotting
  obs_in <- cbind(obs_in, "group" = "obs")
  obs <- data.frame(t(c("model" = "Observations", "group" = "obs_synth", obs)))
  models_in <- cbind(models_in, "group" = "models")
  models <- data.frame(t(c("model" = "Models", "group" = "model_synth", models)))
  synth <- data.frame(t(c("model" = "Synthesis", "group" = "synth", synth)))

  # combine all the data together in one dataframe
  res <- rbind.fill(obs_in, obs, models_in, models, synth)[,c("group", "model", "est", "lower", "upper", "l_wb", "u_wb")]
  for(cnm in c("est", "lower", "upper", "l_wb", "u_wb")) { res[,cnm] <- as.numeric(res[,cnm]) }

  # if only dummy obs, remove
  if(no_obs) {
    # drop all rows that don't relate to models
    res <- res[grepl("model", res$group),]
    sig_obs <- NA
  }

  #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  # reverse any transformations applied
  if(synth_type == "PR") {
    res[,c("est", "lower", "upper", "l_wb", "u_wb")] <- exp(res[,c("est", "lower", "upper", "l_wb", "u_wb")])
    sig_obs <- exp(sig_obs)
    sig_mod <- exp(sig_mod)
    umean <- exp(umean)
  } else if(synth_type == "rel") {
    res[,c("est", "lower", "upper", "l_wb", "u_wb")] <- 100*(exp(res[,c("est", "lower", "upper", "l_wb", "u_wb")])-1)
    sig_obs <- 100*(exp(sig_obs)-1)
    sig_mod <- 100*(exp(sig_mod)-1)
    umean <- 100*(exp(umean)-1)
  }

  return(list(synth_type = synth_type, sig_obs = sig_obs, "chi2/dof" = chi2 / mdof, sig_mod = sig_mod, df = res, uw_mean = umean))
}

# Compare final functions with climate explorer version

https://climexp.knmi.nl/synthesis_202412.cgi?id=62f4b5a82fde776a4c64f0ca33646aa0

In [7]:
for(fnm in list.files("data", full.names = F)) {
    
    fnm <- gsub(".csv", "", fnm)
    # load the data
    df <- read.csv(paste0("data/",fnm,".csv"), row.names = "dataset")

    # split into obs & models, specify the type of synthesis
    obs_in <- df[df$src == "obs",-1]
    models_in <- df[df$src == "models",-1]
    synth_type <- gsub("_","",substr(fnm, nchar(fnm)-2, nchar(fnm)))

        # save in synthesis format
    synth_fnm <- paste0("to_cx/",fnm,".txt")

    obs_cx <- cbind(1980,2020, obs_in, 95,rownames(obs_in))
    model_cx <- cbind(1980,2020, models_in, 95,rownames(models_in))

    colnames(obs_cx) <- colnames(model_cx) <- c("#1", 2:ncol(obs_cx))
    write.table(rbind(obs_cx, model_cx), file = synth_fnm, quote = F, row.names = F)

    # load output from climate explorer
    cx_df <- read.table(paste0("cx_synth/cx-synth_",fnm,".txt"), col.names = c("y1", "y2", "est", "lower", "upper", "l_wb", "u_wb", "group", "model"))

    # run synthesis using R
    synth <- synthesis(obs_in = df[df$src == "obs",-1], models_in = df[df$src == "models",-1], synth_type = synth_type)
    synth_cx <- synthesis_cx(obs_in = df[df$src == "obs",-1], models_in = df[df$src == "models",-1], synth_type = synth_type)

    cx <- readLines(paste0("cx_synth/cx-synth_",fnm,".txt"))

    # get sig_obs parameter from CX output
    if(any(grep("rep", cx))) {
        cx_split <- suppressWarnings(sapply(strsplit(cx[grepl("rep", cx)], " +"), function(x) tryCatch(as.numeric(x))))
        cx_sigobs <- cx_split[!is.na(cx_split)]
    } else {
        cx_sigobs <- NA
    }

    if(any(grepl("model uncertainty is neglected", cx))) {
        cx_sigmod <- NA
    } else {
        cx_split <- suppressWarnings(sapply(strsplit(cx[grepl("model uncertainty", cx)], " +"), function(x) tryCatch(as.numeric(x))))
        cx_sigmod <- cx_split[!is.na(cx_split)]
    } 

    cx <- list("synth_type" = synth_type,
               "sig_obs" = cx_sigobs,
               "chi2/dof" = as.numeric(strsplit(cx[grepl("chi2", cx)], " +")[[1]][5]),
               "sig_mod" = cx_sigmod,
               "df" = read.table(paste0("cx_synth/cx-synth_",fnm,".txt"), col.names = c("y1", "y2", "est", "lower", "upper", "l_wb", "u_wb", "group", "model")))

    res <- rbind("R" = unlist(synth[c("chi2/dof", "sig_obs", "sig_mod")]), "cx" = c(cx[["chi2/dof"]], cx[["sig_obs"]], cx[["sig_mod"]]),
                "R_cx" = unlist(synth_cx[c("chi2/dof", "sig_obs", "sig_mod")]))

                                            
    ########################################################
    synth <- synth_cx
    ########################################################
                                            
                                            
    if(synth_type == "PR") log_axs = "xy" else log_axs = ""
    xylim <- range(unlist(synth$df[,-(1:2)]), na.rm = T)

    pdf(paste0("fig/bounds_",fnm,".pdf"), h = 6, w = 25); {
        layout(matrix(1:5, nrow = 1), widths = c(4,4,5,5,3)/6)
        par(oma = c(0,10,2,0))

        # First base plot
        par(mar = c(3,0.5,3,0.5))
        plot_synthesis(synth, main = "R")
        plot_synthesis(cx, hide_labels = T, main = "Climate explorer")

        mtext(fnm, side = 3, outer = T, font = 2)

        # check coloured bounds for each group
        par(mar = c(3,2,3,0.5))
        plot(synth$df[synth$df$group == "obs", "lower"], cx$df[cx$df$group == "1", "lower"], xlim = xylim, ylim = xylim, log = log_axs, col = "blue", pch = 21, lwd = 2,
            main = "Coloured bounds", xlab = "Computed in R", ylab = "ClimExp")
        points(synth$df[synth$df$group == "obs", "upper"], cx$df[cx$df$group == "1", "upper"], xlim = xylim, ylim = xylim, col = "blue", pch = 3, lwd = 2)

        points(synth$df[synth$df$group == "obs_synth", "lower"], cx$df[cx$df$group == "2", "lower"], xlim = xylim, ylim = xylim, col = "blue4", pch = 21, lwd = 2)
        points(synth$df[synth$df$group == "obs_synth", "upper"], cx$df[cx$df$group == "2", "upper"], xlim = xylim, ylim = xylim, col = "blue4", pch = 3, lwd = 2)

        points(synth$df[synth$df$group == "models", "lower"], cx$df[cx$df$group == "3", "lower"], xlim = xylim, ylim = xylim, col = "red2", pch = 21, lwd = 2)
        points(synth$df[synth$df$group == "models", "upper"], cx$df[cx$df$group == "3", "upper"], xlim = xylim, ylim = xylim, col = "red2", pch = 3, lwd = 2)

        points(synth$df[synth$df$group == "model_synth", "lower"], cx$df[cx$df$group == "4", "lower"], xlim = xylim, ylim = xylim, col = "red4", pch = 21, lwd = 2)
        points(synth$df[synth$df$group == "model_synth", "upper"], cx$df[cx$df$group == "4", "upper"], xlim = xylim, ylim = xylim, col = "red4", pch = 3, lwd = 2)

        points(synth$df[synth$df$group == "synth", "lower"], cx$df[cx$df$group == "5", "lower"], xlim = xylim, ylim = xylim, col = "magenta", pch = 21, lwd = 2)
        points(synth$df[synth$df$group == "synth", "upper"], cx$df[cx$df$group == "5", "upper"], xlim = xylim, ylim = xylim, col = "magenta", pch = 3, lwd = 2)
        abline(0,1, lty = 3) 

        # check white bounds for each group
        plot(synth$df[synth$df$group == "obs", "l_wb"], cx$df[cx$df$group == "1", "l_wb"], xlim = xylim, ylim = xylim, log = log_axs, col = "blue", pch = 21, lwd = 2,
            main = "White bounds", xlab = "Computed in R", ylab = "ClimExp")
        points(synth$df[synth$df$group == "obs", "u_wb"], cx$df[cx$df$group == "1", "u_wb"], xlim = xylim, ylim = xylim, col = "blue", pch = 3, lwd = 2)

        points(synth$df[synth$df$group == "models", "l_wb"], cx$df[cx$df$group == "3", "l_wb"], xlim = xylim, ylim = xylim, col = "red2", pch = 21, lwd = 2)
        points(synth$df[synth$df$group == "models", "u_wb"], cx$df[cx$df$group == "3", "u_wb"], xlim = xylim, ylim = xylim, col = "red2", pch = 3, lwd = 2)

        points(synth$df[synth$df$group == "synth", "l_wb"], cx$df[cx$df$group == "5", "l_wb"], xlim = xylim, ylim = xylim, col = "magenta", pch = 21, lwd = 2)
        points(synth$df[synth$df$group == "synth", "u_wb"], cx$df[cx$df$group == "5", "u_wb"], xlim = xylim, ylim = xylim, col = "magenta", pch = 3, lwd = 2)
        abline(0,1, lty = 3) 


        legend("bottomright", c("Lower bound", "Upper bound"), pch = c(21,3))

        # plot table of results
        frame()
        vps <- baseViewports()
        pushViewport(vps$inner, vps$figure, vps$plot)

        grob <- tableGrob(round(res,3),  vp = viewport(x = 0.1, y = 0.75))
        grid.draw(grob)

    }; dev.off()
}

In [5]:
# save in synthesis format
synth_fnm <- paste0("to_cx/",fnm,".txt")

obs_cx <- cbind(1980,2020, obs_in, 95,rownames(obs_in))
model_cx <- cbind(1980,2020, models_in, 95,rownames(models_in))

colnames(obs_cx) <- colnames(model_cx) <- c("#1", 2:ncol(obs_cx))
write.table(rbind(obs_cx, model_cx), file = synth_fnm, quote = F, row.names = F)

# load output from climate explorer
cx_df <- read.table(paste0("cx_synth/cx-synth_",fnm,".txt"), col.names = c("y1", "y2", "est", "lower", "upper", "l_wb", "u_wb", "group", "model"))

# run synthesis using R
synth <- synthesis(obs_in = df[df$src == "obs",-1], models_in = df[df$src == "models",-1], synth_type = synth_type)

# synthesis using 2sigma instead of 1.96sigma
synth2 <- synthesis2(obs_in = df[df$src == "obs",-1], models_in = df[df$src == "models",-1], synth_type = synth_type)

cx <- readLines(paste0("cx_synth/cx-synth_",fnm,".txt"))

# get sig_obs parameter from CX output
if(any(grep("rep", cx))) {
    cx_split <- suppressWarnings(sapply(strsplit(cx[grepl("rep", cx)], " +"), function(x) tryCatch(as.numeric(x))))
    cx_sigobs <- cx_split[!is.na(cx_split)]
} else {
    cx_sigobs <- NA
}
                                        
if(any(grepl("model uncertainty is neglected", cx))) {
    cx_sigmod <- NA
} else {
    cx_split <- suppressWarnings(sapply(strsplit(cx[grepl("model uncertainty", cx)], " +"), function(x) tryCatch(as.numeric(x))))
    cx_sigmod <- cx_split[!is.na(cx_split)]
} 
                                        
cx <- list("synth_type" = synth_type,
           "sig_obs" = cx_sigobs,
           "chi2/dof" = as.numeric(strsplit(cx[grepl("chi2", cx)], " +")[[1]][5]),
           "sig_mod" = cx_sigmod,
           "df" = read.table(paste0("cx_synth/cx-synth_",fnm,".txt"), col.names = c("y1", "y2", "est", "lower", "upper", "l_wb", "u_wb", "group", "model")))
                                        
res <- rbind("R" = unlist(synth[c("chi2/dof", "sig_obs", "sig_mod")]), "cx" = c(cx[["chi2/dof"]], cx[["sig_obs"]], cx[["sig_mod"]]),
             "R2" = unlist(synth2[c("chi2/dof", "sig_obs", "sig_mod")]))

if(synth_type == "PR") log_axs = "xy" else log_axs = ""
xylim <- range(unlist(synth$df[,-(1:2)]), na.rm = T)

pdf(paste0("fig/bounds_",fnm,".pdf"), h = 6, w = 25); {
    layout(matrix(1:5, nrow = 1), widths = c(4,4,5,5,3)/6)
    par(oma = c(0,10,2,0))

    # First base plot
    par(mar = c(3,0.5,3,0.5))
    plot_synthesis(synth, main = "R")
    plot_synthesis(cx, hide_labels = T, main = "Climate explorer")
    
    mtext(fnm, side = 3, outer = T, font = 2)

    # check coloured bounds for each group
    par(mar = c(3,2,3,0.5))
    plot(synth$df[synth$df$group == "obs", "lower"], cx$df[cx$df$group == "1", "lower"], xlim = xylim, ylim = xylim, log = log_axs, col = "blue", pch = 21, lwd = 2,
        main = "Coloured bounds", xlab = "Computed in R", ylab = "ClimExp")
    points(synth$df[synth$df$group == "obs", "upper"], cx$df[cx$df$group == "1", "upper"], xlim = xylim, ylim = xylim, col = "blue", pch = 3, lwd = 2)

    points(synth$df[synth$df$group == "obs_synth", "lower"], cx$df[cx$df$group == "2", "lower"], xlim = xylim, ylim = xylim, col = "blue4", pch = 21, lwd = 2)
    points(synth$df[synth$df$group == "obs_synth", "upper"], cx$df[cx$df$group == "2", "upper"], xlim = xylim, ylim = xylim, col = "blue4", pch = 3, lwd = 2)

    points(synth$df[synth$df$group == "models", "lower"], cx$df[cx$df$group == "3", "lower"], xlim = xylim, ylim = xylim, col = "red2", pch = 21, lwd = 2)
    points(synth$df[synth$df$group == "models", "upper"], cx$df[cx$df$group == "3", "upper"], xlim = xylim, ylim = xylim, col = "red2", pch = 3, lwd = 2)

    points(synth$df[synth$df$group == "model_synth", "lower"], cx$df[cx$df$group == "4", "lower"], xlim = xylim, ylim = xylim, col = "red4", pch = 21, lwd = 2)
    points(synth$df[synth$df$group == "model_synth", "upper"], cx$df[cx$df$group == "4", "upper"], xlim = xylim, ylim = xylim, col = "red4", pch = 3, lwd = 2)

    points(synth$df[synth$df$group == "synth", "lower"], cx$df[cx$df$group == "5", "lower"], xlim = xylim, ylim = xylim, col = "magenta", pch = 21, lwd = 2)
    points(synth$df[synth$df$group == "synth", "upper"], cx$df[cx$df$group == "5", "upper"], xlim = xylim, ylim = xylim, col = "magenta", pch = 3, lwd = 2)
    abline(0,1, lty = 3) 

    # check white bounds for each group
    plot(synth$df[synth$df$group == "obs", "l_wb"], cx$df[cx$df$group == "1", "l_wb"], xlim = xylim, ylim = xylim, log = log_axs, col = "blue", pch = 21, lwd = 2,
        main = "White bounds", xlab = "Computed in R", ylab = "ClimExp")
    points(synth$df[synth$df$group == "obs", "u_wb"], cx$df[cx$df$group == "1", "u_wb"], xlim = xylim, ylim = xylim, col = "blue", pch = 3, lwd = 2)

    points(synth$df[synth$df$group == "models", "l_wb"], cx$df[cx$df$group == "3", "l_wb"], xlim = xylim, ylim = xylim, col = "red2", pch = 21, lwd = 2)
    points(synth$df[synth$df$group == "models", "u_wb"], cx$df[cx$df$group == "3", "u_wb"], xlim = xylim, ylim = xylim, col = "red2", pch = 3, lwd = 2)

    points(synth$df[synth$df$group == "synth", "l_wb"], cx$df[cx$df$group == "5", "l_wb"], xlim = xylim, ylim = xylim, col = "magenta", pch = 21, lwd = 2)
    points(synth$df[synth$df$group == "synth", "u_wb"], cx$df[cx$df$group == "5", "u_wb"], xlim = xylim, ylim = xylim, col = "magenta", pch = 3, lwd = 2)
    abline(0,1, lty = 3) 

    
    legend("bottomright", c("Lower bound", "Upper bound"), pch = c(21,3))

    # plot table of results
    frame()
    vps <- baseViewports()
    pushViewport(vps$inner, vps$figure, vps$plot)
    
#     grid.table(round(res, 3))

    grob <-  tableGrob(round(res,3),  vp = viewport(x = 0.1, y = 0.75))
    grid.draw(grob)
    
     
}; dev.off()

ERROR: Error in synthesis2(obs_in = df[df$src == "obs", -1], models_in = df[df$src == : could not find function "synthesis2"
